In [37]:
# import libraries
import numpy as np
import pandas as pd
import itertools
from datetime import datetime
import time
import math
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import roc_auc_score
from lightgbm import LGBMClassifier
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import make_pipeline, Pipeline
from tqdm import tqdm
from contextlib import contextmanager
import gc
import re
import string

# plotting libraries
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

# Ignore useless warnings 
import warnings
warnings.filterwarnings(action="ignore")

# set to show all rows and columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


sns.set_style('darkgrid') # darkgrid, white grid, dark, white and ticks
plt.rc('axes', titlesize=18)     # fontsize of the axes title
plt.rc('axes', labelsize=14)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=13)    # fontsize of the tick labels
plt.rc('ytick', labelsize=13)    # fontsize of the tick labels
plt.rc('legend', fontsize=13)    # legend fontsize
plt.rc('font', size=13)          # controls default text sizes

# Product recommendation
please run all function definition cells first, then load in main cell

Make 3 models in total and create ensemble

In [35]:
# -------------------------- Main ------------------------
# load in data set with piplines
# use month 4, 5, 6 to predict 7
# 3 models total: 
#     1. 6 predict 7
#     2. 5+6 predict 7
#     3. 4+5+6 (also need to create lag variables from 4 to 6) predict 7
# depending on the months length we change length of test set as well
# then ensemble
# get months dataset with flagged numbers
df_main, r, s = main_month(r"C:\Users\120200\Desktop\Product Recommendation-Tiger\Train\B360_202106_10Percent_DawhoOnly.csv", 
                           r"C:\Users\120200\Desktop\Product Recommendation-Tiger\Train\B360_202107_10Percent_DawhoOnly.csv", 
                           '6and7')
df_5 = prev_month(r"C:\Users\120200\Desktop\Product Recommendation-Tiger\Train\B360_202105_10Percent_DawhoOnly.csv",
                  r"C:\Users\120200\Desktop\Product Recommendation-Tiger\Train\B360_202106_10Percent_DawhoOnly.csv",
                  r, s, '5and6')
df_4 = prev_month(r"C:\Users\120200\Desktop\Product Recommendation-Tiger\Train\B360_202104_10Percent_DawhoOnly.csv",
                  r"C:\Users\120200\Desktop\Product Recommendation-Tiger\Train\B360_202105_10Percent_DawhoOnly.csv",
                  r, s, '4and5')

100%|███████████████████████████████████████████████████████████████████████████████| 522/522 [00:01<00:00, 505.47it/s]


Main month execution time: 11.042273998260498 seconds
exception catched: 薪轉代理編號取號日
encoding execution time: 12.124212741851807 seconds
Dataframe shape:  (46584, 719)
total execution time: 35.11506462097168 seconds
-----------------------------------------------------------------------
Previous month execution time: 3.8519320487976074 seconds
exception catched: 薪轉代理編號取號日
encoding execution time: 12.488150358200073 seconds
Dataframe shape:  (47666, 721)
total execution time: 24.42594075202942 seconds
-----------------------------------------------------------------------
Previous month execution time: 3.688673734664917 seconds
exception catched: 薪轉代理編號取號日
encoding execution time: 11.988296270370483 seconds
Dataframe shape:  (46684, 722)
total execution time: 23.70610022544861 seconds
-----------------------------------------------------------------------


In [ ]:
# merge and flagging datasets
# first dataset is just df_main
df_6pred7 = df_main
df_56pred7 = merge_and_flag(df_main, df_5, 6, 5)
df_456pred_7 = merge_and_flag(df_56pred7, df_4, 6, 4)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3418, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-41-c43b453934ca>", line 4, in <module>
    df_56pred7 = merge_and_flag(df_main, df_5, 6, 5)
  File "<ipython-input-28-4e3fbe2925fb>", line 10, in merge_and_flag
    df_total = pd.merge(df_primary, df_secondary, on='身分證字號', suffixes=('', '_' + month_2), how='left')
  File "C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\reshape\merge.py", line 83, in merge
    return op.get_result()
  File "C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\reshape\merge.py", line 658, in get_result
    copy=self.copy,
  File "C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\internals\managers.py", line 2054, in concatenate_block_managers
    concatenate_join_units(join_units, concat_axis, copy=copy),
  File "C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\intern

In [17]:
print(df_56pred7.shape)
print(df_456pred_7.shape)

(46624, 2164)
(46624, 3608)


In [ ]:
# Running most simple model to get feature importances
# drop all columns with feature importance = 0
# and it seems like we need a larger sample
with timer("Run LightGBM"):
    feat_importance = kfold_lightgbm_sklearn(df_main, '信貸(41_42)增加記號6and7')
    print(feat_importance)

Train/valid shape: (46584, 723)
ERROR! Session/line number was not unique in database. History logging moved to new session 106
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[2]	training's auc: 0.743665	training's binary_logloss: 0.0096507	valid_1's auc: 0.0749249	valid_1's binary_logloss: 0.0095403
Fold  1 AUC : 0.074925
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[2]	training's auc: 0.659227	training's binary_logloss: 0.00913149	valid_1's auc: 0.154143	valid_1's binary_logloss: 0.0141047
Fold  2 AUC : 0.154143
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[9]	training's auc: 0.937149	training's binary_logloss: 0.0185659	valid_1's auc: 0.814258	valid_1's binary_logloss: 0.0448088
Fold  3 AUC : 0.814258
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[2]	training's auc: 0.705979	traini

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3418, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-41-894d6a00d8cc>", line 5, in <module>
    feat_importance = kfold_lightgbm_sklearn(df_main, '信貸(41_42)增加記號6and7')
  File "<ipython-input-40-c3365764da3d>", line 40, in kfold_lightgbm_sklearn
    eval_metric='auc', verbose=400, early_stopping_rounds= EARLY_STOPPING)
  File "C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\sklearn.py", line 895, in fit
    callbacks=callbacks, init_model=init_model)
  File "C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\sklearn.py", line 688, in fit
    callbacks=callbacks, init_model=init_model)
  File "C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py", line 228, in train
    booster = Booster(params=params, train_set=train_set)
  File "C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\basic.py", line 2229, in __

In [40]:
with timer("Run LightGBM"):
    feat_importance = kfold_lightgbm_sklearn(df_56pred7, '信貸(41_42)增加記號6and7')
    print(feat_importance)

Train/valid shape: (46624, 2164)
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[2]	training's auc: 0.565989	training's binary_logloss: 0.0153228	valid_1's auc: 0.508827	valid_1's binary_logloss: 0.0106254
Fold  1 AUC : 0.508827
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[2]	training's auc: 0.665646	training's binary_logloss: 0.0155417	valid_1's auc: 0.255044	valid_1's binary_logloss: 0.0142014
Fold  2 AUC : 0.255044
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1]	training's auc: 0.997608	training's binary_logloss: 0.0369006	valid_1's auc: 0.611639	valid_1's binary_logloss: 0.0545966
Fold  3 AUC : 0.611639
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[2]	training's auc: 0.666951	training's binary_logloss: 0.0139221	valid_1's auc: 0.459299	valid_1's binary_logloss: 0.0232498
Fold  

In [41]:
with timer("Run LightGBM"):
    feat_importance = kfold_lightgbm_sklearn(df_456pred_7, '信貸(41_42)增加記號6and7')
    print(feat_importance)

Train/valid shape: (46624, 3608)
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[2]	training's auc: 0.732212	training's binary_logloss: 0.0121323	valid_1's auc: 0.364429	valid_1's binary_logloss: 0.0117358
Fold  1 AUC : 0.364429
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[2]	training's auc: 0.798622	training's binary_logloss: 0.0125953	valid_1's auc: 0.247612	valid_1's binary_logloss: 0.0163139
Fold  2 AUC : 0.247612
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[2]	training's auc: 0.599529	training's binary_logloss: 0.0180257	valid_1's auc: 0.62331	valid_1's binary_logloss: 0.0264599
Fold  3 AUC : 0.623310
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[6]	training's auc: 0.898527	training's binary_logloss: 0.0153171	valid_1's auc: 0.462787	valid_1's binary_logloss: 0.0143675
Fold  4

In [69]:
# lets drop all columns with feat_importance = 0 and see if that helps our model
feat_gain = feat_importance[feat_importance['gain'] == 0] 
dropcols = feat_gain['feature'].tolist()
droplist = []
for i in dropcols:
    i = i.replace('q', ',')
    droplist.append(i)

df_456pred_7.drop(droplist, axis=1, inplace=True)

In [70]:
# second try
# overall improved a little (0.5) now we try making more moves on feature engineering
with timer("Run LightGBM"):
    feat_importance = kfold_lightgbm_sklearn(df_456pred_7, '當月基金申購金額增加記號6and7')
    print(feat_importance)

Train/valid shape: (46624, 677)
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[61]	training's auc: 0.984131	training's binary_logloss: 0.000855231	valid_1's auc: 0.473859	valid_1's binary_logloss: 0.0213008
Fold  1 AUC : 0.473859
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[24]	training's auc: 1	training's binary_logloss: 0.000838714	valid_1's auc: 0.812623	valid_1's binary_logloss: 0.0175009
Fold  2 AUC : 0.812623
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[7]	training's auc: 0.951712	training's binary_logloss: 0.00785952	valid_1's auc: 0.795149	valid_1's binary_logloss: 0.0375297
Fold  3 AUC : 0.795149
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[9]	training's auc: 0.919324	training's binary_logloss: 0.0080928	valid_1's auc: 0.661558	valid_1's binary_logloss: 0.0163463
Fold  4

In [ ]:
# Running most simple model to get feature importances
# drop all columns with feature importance = 0
# and it seems like we need a larger sample
with timer("Run LightGBM"):
    feat_importance = kfold_lightgbm_sklearn(df_6pred7, '當月基金申購金額增加記號6and7')
    print(feat_importance)

In [40]:
# ------- lgbm model ---------------------------------------
def kfold_lightgbm_sklearn(df, target_col):
    print("Train/valid shape: {}".format(df.shape))
    # LightGBM does not support special chars hence we chage commas into q
    # reference: https://github.com/microsoft/LightGBM/issues/2478 
    df = df.rename(columns = lambda x:re.sub('\,', 'q', x))
    targets = ['一般房貸(36)增加記號6and7', 
               '信貸(41_42)增加記號6and7', 
               '理財房貸(34_35_47)增加記號6and7', 
               '當月基金申購金額增加記號6and7']
    target = target_col
    del_features = ['身分證字號']
    df = df.drop(del_features, axis=1)
    predictors = df.drop(targets, axis = 1).columns.tolist()

    if not STRATIFIED_KFOLD:
        folds = KFold(n_splits= NUM_FOLDS, shuffle=True, random_state= RANDOM_SEED)
    else:
        folds = StratifiedKFold(n_splits= NUM_FOLDS, shuffle=True, random_state= RANDOM_SEED)

    # Hold oof predictions, feature importance and training/valid auc
    oof_preds = np.zeros(df.shape[0])
    importance_df = pd.DataFrame()
    eval_results = dict()

    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(df[predictors], df[target])):
        train_x, train_y = df[predictors].iloc[train_idx], df[target].iloc[train_idx]
        valid_x, valid_y = df[predictors].iloc[valid_idx], df[target].iloc[valid_idx]
        
        params = {'random_state': RANDOM_SEED}
        clf = LGBMClassifier(**{**params, **LIGHTGBM_PARAMS})
        
#         # implement SMOTE
#         pipeline = (SMOTE(sampling_strategy='minority'), clf)
        
#         pipeline.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)],
#                      eval_metric='auc', verbose=400, early_stopping_rounds= EARLY_STOPPING)
                
        clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)],
                     eval_metric='auc', verbose=400, early_stopping_rounds= EARLY_STOPPING)

        oof_preds[valid_idx] = clf.predict_proba(valid_x, num_iteration=clf.best_iteration_)[:, 1]
#         sub_preds += clf.predict_proba(test[predictors], num_iteration=clf.best_iteration_)[:, 1] / folds.n_splits

        # Feature importance by GAIN and SPLIT
        fold_importance = pd.DataFrame()
        fold_importance["feature"] = predictors
        fold_importance["gain"] = clf.booster_.feature_importance(importance_type='gain')
        fold_importance["split"] = clf.booster_.feature_importance(importance_type='split')
        importance_df = pd.concat([importance_df, fold_importance], axis=0)
        eval_results['train_{}'.format(n_fold+1)]  = clf.evals_result_['training']['auc']
        eval_results['valid_{}'.format(n_fold+1)] = clf.evals_result_['valid_1']['auc']

        print('Fold %2d AUC : %.6f' % (n_fold + 1, roc_auc_score(valid_y, oof_preds[valid_idx])))
        del clf, train_x, train_y, valid_x, valid_y
        gc.collect()

    print('Full AUC score %.6f' % roc_auc_score(df[target], oof_preds))
#     test['TARGET'] = sub_preds.copy()

    # Get the average feature importance between folds
    mean_importance = importance_df.groupby('feature').mean().reset_index()
    mean_importance.sort_values(by= 'gain', ascending=False, inplace=True)
    return mean_importance



ERROR! Session/line number was not unique in database. History logging moved to new session 105


In [19]:
def main_month(path1, path2, postfix):
    starttime = time.time()
    # get newest months data and preprocess
    df, r, s, m = get_main_month(path1)
    df = format_dates(df, m)
    df = encode_data(df)
    df = add_feature(df)
    # at this point only null values are from add feature, where numbers are divided by zero
    # we fill them with 0 and we have a perfect dataset
    df.fillna(0, inplace=True)
    df = make_y_value(df, path2, postfix)
    print("total execution time: %s seconds" % (time.time() - starttime))
    print("-----------------------------------------------------------------------")
    return df, r, s 

In [20]:
# try getting previous month (may)
# basically same as anbove but with different dataset
def prev_month(path1, path2, r, s, postfix):
    starttime = time.time()
    df_5, m_5 = get_prev_month(path1, s, r)
    df_5 = format_dates(df_5, m_5)
    df_5 = encode_data(df_5)
    df_5 = add_feature(df_5)
    df_5.fillna(0, inplace=True)
    #  this creates columns we can use as lag-variables! Might be usefull.
    df_5 = make_y_value(df_5, path2, postfix)  
    print("total execution time: %s seconds" % (time.time() - starttime))
    print("-----------------------------------------------------------------------")
    return df_5

In [21]:
# Make y values
# 放款總餘額  一般房貸(36)            float64
# 放款總餘額  信貸(41_42)           float64
# 放款總餘額  理財房貸(34_35_47)      float64
# 當月基金申購金額
# get next month (7)
def make_y_value(df, next_month_path, postfix):
    df_next = pd.read_csv(next_month_path)
    naming = postfix

    merge_cols = ['身分證字號', 
                  '放款總餘額  一般房貸(36)', 
                  '放款總餘額  信貸(41_42)', 
                  '放款總餘額  理財房貸(34_35_47)', 
                  '當月基金申購金額']
    df = pd.merge(df, df_next[merge_cols], on='身分證字號', suffixes=('', '_Y'), how='left')
    
    df["一般房貸(36)增加記號" + naming] = df['放款總餘額  一般房貸(36)_Y'] - df['放款總餘額  一般房貸(36)']
    df["信貸(41_42)增加記號" + naming] = df['放款總餘額  信貸(41_42)_Y'] - df['放款總餘額  信貸(41_42)']
    df["理財房貸(34_35_47)增加記號" + naming] = df['放款總餘額  理財房貸(34_35_47)_Y'] - df['放款總餘額  理財房貸(34_35_47)']
    df["當月基金申購金額增加記號" + naming] = df['當月基金申購金額_Y'] - df['當月基金申購金額']
    
    drop_used_cols = ['放款總餘額  一般房貸(36)_Y', 
                      '放款總餘額  信貸(41_42)_Y', 
                      '放款總餘額  理財房貸(34_35_47)_Y', 
                      '當月基金申購金額_Y']
    df = df.drop(drop_used_cols, axis=1, )
    # change y vlaues to 0 and 1 since we want probability
    # everyting greater than 0 is 1, less will be zero
    y_cols = ["一般房貸(36)增加記號" + naming,
              '信貸(41_42)增加記號' + naming,
              '理財房貸(34_35_47)增加記號' + naming,
              '當月基金申購金額增加記號' + naming]
    for col in y_cols:
        current_column = df[col].to_list()
        for i, v in enumerate(current_column):
            if v <= 0:
                current_column[i] = 0
            else:
                current_column[i] = 1
        df[col] = pd.Series(current_column)
    return df

In [22]:
# ----------------- Month pipeline ------------------------------------#
# main purpose is to clean out useless data and change date objects into integer numbers

def get_main_month(path):
    df = pd.read_csv(path)
    starttime = time.time()
    
    # drop some useless or unusable columns
    droplist = ['員工記號',
                '金控員工記號(非銀行)',
                '分行別',
                'CIFAOCODE',
                '薪轉代理編號',
                '主戶ID',
                '負責人身份證字號'
               ]
    df.drop(droplist, axis=1, inplace=True)
    
    # for 數位帳戶記號, there are sometimes 0's, which means this customer 
    # closed their DW account, we will find those index's and delete these rows
    # removed index: customers that left our service
    cancelled = df.index[df['數位帳戶記號'] == 0].tolist()
    drop = df.index[df['性別'] == 0].tolist()
    cancelled = cancelled + drop
    # sex has a 0 for 外僑
    df.drop(cancelled, axis=0, inplace=True)
    df = df.reset_index(drop=True)
    
    # first remove columns with identical values
    # note this step will remove the date column for this month so we need to record that
    # and change it to datetime object
    dataMonth = pd.DataFrame(pd.to_datetime(df['資料年月'], format='%Y%m'))
    df_dict = df.to_dict()
    singleVariableCols = []
    for key in df_dict:
        if (len(df[key].value_counts(dropna=False)) == 1):
            singleVariableCols.append(key)
    df.drop(singleVariableCols, axis=1, inplace=True)
    
    # find list of repeated variables and delete those columns
    # multiple columns with the same values do not provide extra information
    repeatedCols = []
    cols = df.columns.to_list()
    for i_1, v_1 in enumerate(tqdm(cols)):
        for i_2, v_2 in enumerate(cols[i_1+1:]):
            if (df_dict[v_1] == df_dict[v_2]):
                repeatedCols.append(v_2)
    df.drop(repeatedCols, axis=1, inplace=True)
    print("Main month execution time: %s seconds" % (time.time() - starttime))
    return df, repeatedCols, singleVariableCols, dataMonth

In [23]:
# get other months but need to drop same columns as newest month
def get_prev_month(path, repeatedCols, singleVariableCols):
    # drop some useless or unusable columns
    starttime = time.time()
    droplist = ['員工記號',
                '金控員工記號(非銀行)',
                '分行別',
                'CIFAOCODE',
                '薪轉代理編號',
                '主戶ID',
                '負責人身份證字號'
               ]
    
    df = pd.read_csv(path)

    dataMonth = pd.DataFrame(pd.to_datetime(df['資料年月'], format='%Y%m')) 
    
    # for 數位帳戶記號, there are sometimes 0's, which means this customer 
    # closed their DW account, we will find those index's and delete these rows
    # removed index: customers that left our service
    cancelled = df.index[df['數位帳戶記號'] == 0].tolist()
    # sex has a 0 for 外僑
    drop = df.index[df['性別'] == 0].tolist()
    removed_index = cancelled + drop
    df.drop(removed_index, axis=0, inplace=True)
    df = df.reset_index(drop=True)
    
    df.drop(droplist, axis=1, inplace=True)
    df.drop(singleVariableCols, axis=1, inplace=True)
    df.drop(repeatedCols, axis=1, inplace=True)
    print("Previous month execution time: %s seconds" % (time.time() - starttime))
    return df, dataMonth

In [24]:
def format_dates(df, dataMonth):
    # date columns
    # change to datetime object and get difference of days up untill today
    # last month date of 資料年月 
    # Note: leaving days that are set as 1990-01-01 now, since it will become a large number which w
    #       wouldnt affect our data that much
    dataMonth = dataMonth
    dateCols = ['最初開戶日', 
                '最近交易日',
                '最後通聯日',
                '存單最近到期日',
                '存款最近交易日',
                '存款開戶日',
                '信用戶開戶日期',
                '信託帳戶開戶日',
                '信託第一筆申購日',
                '信託最近交易日',
                '最近一次應繳日(要保人)',
                '證券最近交易日',
                '證券開戶日',
                '房貸開戶日',
                '放款最近交易日',
                '放款結清戶日期',
                '放款開戶日',
                '信貸開戶日',
                '信用卡開戶日',
                '薪轉代理編號取號日',   # this column was deleted as it contains one value only so we catch exceptions
                'Line_第一次綁定時間',
                'Line_最後綁定時間',
                'Line_第一次加好友時間',
                'Line_好友狀態異動時間',
                '存款最早開戶日']         # Note only this column needs a different format
    # since there might be missing columns when deleting we will pass those errors
    for i in dateCols:
        # first we catch if any date columns have been deleted and skip those
        try: 
            df[i]
        except:
            print("exception catched: " + str(i))
            continue
        # this column needs special formatting
            if (i == '存款最早開戶日'):
                try:
                    df[i] = pd.to_datetime(df[i], format='%Y%m%d')
                    # we change delete from current month and change dtype to int
                    df[i] = (dataMonth['資料年月'] - df[i])
                    df[i] = df[i].dt.days
                except:
                    print("存款最早開戶日 is all 0's type int, this month's 存款最早開戶日 should be replaced by next months data")
                    pass
        else:
            # same as above
            df[i] = pd.to_datetime(df[i], format='%Y-%m-%d')
            df[i] = (dataMonth['資料年月'] - df[i])
            df[i] = df[i].dt.days
    return df

In [25]:
def add_feature(df):
    # Finally lets combine the 
    # add extra features, specifically more features on time
    df['客戶年收入/最初開戶日'] = df['客戶年收入'] / df['最初開戶日']
    df['客戶年收入/往來期間(月)'] = df['客戶年收入'] / df['往來期間(月)']
    df['最近交易日-最後通聯日'] = df['最近交易日'] - df['最後通聯日']
    # 台外幣總存款
    df['台外幣總存款/最初開戶日'] = df['台外幣總存款'] / df['最初開戶日']
    df['台外幣總存款/往來期間(月)'] = df['台外幣總存款'] / df['往來期間(月)']
    df['台外幣總存款/客戶年收入'] = df['台外幣總存款'] / df['客戶年收入']
    # 台幣支存總餘額
    df['台幣支存總餘額/最初開戶日'] = df['台幣支存總餘額'] / df['最初開戶日']
    df['台幣支存總餘額/往來期間(月)'] = df['台幣支存總餘額'] / df['往來期間(月)']
    df['台幣支存總餘額/客戶年收入'] = df['台幣支存總餘額'] / df['客戶年收入']
    # 台幣定存總餘額
    df['台幣定存總餘額/最初開戶日'] = df['台幣定存總餘額'] / df['最初開戶日']
    df['台幣定存總餘額/往來期間(月)'] = df['台幣定存總餘額'] / df['往來期間(月)']
    df['台幣定存總餘額/客戶年收入'] = df['台幣定存總餘額'] / df['客戶年收入']
    # 台幣活期性總存款
    df['台幣活期性總存款/最初開戶日'] = df['台幣活期性總存款'] / df['最初開戶日']
    df['台幣活期性總存款/往來期間(月)'] = df['台幣活期性總存款'] / df['往來期間(月)']
    df['台幣活期性總存款/客戶年收入'] = df['台幣活期性總存款'] / df['客戶年收入']
    # 外幣總存款
    df['外幣總存款/最初開戶日'] = df['外幣總存款'] / df['最初開戶日']
    df['外幣總存款/往來期間(月)'] = df['外幣總存款'] / df['往來期間(月)']
    df['外幣總存款/客戶年收入'] = df['外幣總存款'] / df['客戶年收入']
    # 台外幣總存款 / 存單最近到期日
    df['台外幣總存款/存單最近到期日'] = df['台外幣總存款'] / df['存單最近到期日']
    df['台幣支存總餘額/存單最近到期日'] = df['台幣支存總餘額'] / df['存單最近到期日']
    df['台幣定存總餘額/存單最近到期日'] = df['台幣定存總餘額'] / df['存單最近到期日']
    df['台幣活期性總存款/存單最近到期日'] = df['台幣活期性總存款'] / df['存單最近到期日']
    df['外幣總存款/存單最近到期日'] = df['外幣總存款'] / df['存單最近到期日']
    # 存單最近到期金額 / 存單最近到期日
    df['存單最近到期金額/存單最近到期日'] = df['存單最近到期金額'] / df['存單最近到期金額']
    df['台外幣總存款/存單最近到期金額'] = df['台外幣總存款'] / df['存單最近到期金額']
    df['台幣支存總餘額/存單最近到期金額'] = df['台幣支存總餘額'] / df['存單最近到期金額']
    df['台幣定存總餘額/存單最近到期金額'] = df['台幣定存總餘額'] / df['存單最近到期金額']
    df['台幣活期性總存款/存單最近到期金額'] = df['台幣活期性總存款'] / df['存單最近到期金額']
    df['外幣總存款/存單最近到期金額'] = df['外幣總存款'] / df['存單最近到期金額']
    # 最後通聯日
    df['存單最近到期金額/最後通聯日'] = df['存單最近到期金額'] / df['最後通聯日']
    df['台外幣總存款/最後通聯日'] = df['台外幣總存款'] / df['最後通聯日']
    df['台幣支存總餘額/最後通聯日'] = df['台幣支存總餘額'] / df['最後通聯日']
    df['台幣定存總餘額/最後通聯日'] = df['台幣定存總餘額'] / df['最後通聯日']
    df['台幣活期性總存款/最後通聯日'] = df['台幣活期性總存款'] / df['最後通聯日']
    df['外幣總存款/最後通聯日'] = df['外幣總存款'] / df['最後通聯日']   
    print("Dataframe shape: ", df.shape)
    return df

In [26]:
# --------------- Categorical Data Pipeline ----------------------------------
def encode_data(df):
    starttime = time.time()
    # combine job descritpion columns where ELOAN > 信用卡職位別 > 客戶職稱 if both contain values
    # 客戶職稱                       object
    # ELOAN職稱                    object
    # 信用卡職位別                     object
    jd_eloan = df['ELOAN職稱'].to_list()
    jd_card = df['信用卡職位別'].to_list()
    jd_customer = df['客戶職稱'].tolist()
    for i in np.arange(0, len(jd_eloan), 1):
        if isinstance(jd_eloan[i],str):
            continue
        elif isinstance(jd_card[i], str):
            jd_eloan[i] = jd_card[i]
        elif isinstance(jd_customer[i], str):
            jd_eloan[i] = jd_customer[i]
        else:
            continue
    df.drop(['ELOAN職稱', '信用卡職位別', '客戶職稱'], axis=1, inplace=True)
    df['職稱'] = pd.Series(jd_eloan)
    df = pd.get_dummies(df, columns = ['職稱'])
    
    # this columns is weird, it has type str and type int 2, and a '-'
    # will change all to type int and - to zero then get dummy
    test = df['對帳單寄送記號'].to_list()
    for i, v in enumerate(test):
        try:
            df['對帳單寄送記號'][i] = int(v)
        except:
            df['對帳單寄送記號'][i] = 0
    df = pd.get_dummies(df, columns = ['對帳單寄送記號'])
    
    dummies = ['客戶行業別',
              '信用評等資料-中擔餘額',
              '信用評等資料-長擔餘額',
              '信用評等資料-信貸餘額',
              '電話１(戶籍)失聯記號',
              '電話２(住家)失聯記號',
              '電話４(公司)失聯記號',
              '電話５(行動)失聯記號',
              '電話８(其他)失聯記號',
              '關係戶記號',
              'Line_互動程度',
              '婚姻',
              '住宅記號',
              '專業投資人記號',
              '企金行業別', 
              '企審催收記號']
    for i in dummies:
        try:
            df[i] = df[i].astype('Int64')
        except:
            continue
    df = pd.get_dummies(df,columns = dummies, dummy_na=True)

    YN_to_int_cols = ['信用卡VIP客戶註記',
                      '豐掌櫃會員記號',
                      '人民幣存款註記',
                      '歐元存款註記',
                      '澳幣存款註記',
                      '南非幣存款註記',
                      '保管箱記號',
                      '通訊地址失聯記號',
                      '定時定額記號',
                      '單筆申購記號',
                      '自動扣款記號',
                      '信用卡轉呆記號',
                      '外幣存款記號']
    # label encode Y and N's to 1 and 0
    for col in YN_to_int_cols:
        df[col], uniques = pd.factorize(df[col], sort=True)
    # Change null to str. and other values to string type then get dummies
    null_to_str_cols = ['投資屬性',
                         '網易收會員記號', 
                         '銀行推介(證券端記號)記號',
                         '個金放款型客戶',
                         '保險-要保人記號',
                         '保險-被保人記號',
                         '家庭保障險申購記號(要保人)',
                         '家庭保障險申購記號(被保人)',
                         '產險-要保人記號',
                         '信用卡逾期紀錄',
                         '信用循環戶記號',
                        ]
    for i in null_to_str_cols:
        try:
            df[i] = df[i].astype('Int64')
        except:
            continue
    df = pd.get_dummies(df,columns = null_to_str_cols, dummy_na=True)    
        
    has_value_to_cat = ['薪轉公司統編', 
                         '主經管人員', 
                        ]
    for col in has_value_to_cat:
        current_column = df[col].to_list()
        for i, v in enumerate(current_column):
            if math.isnan(v):
                current_column[i] = 0
            else:
                current_column[i] = 1
        df[col] = pd.Series(current_column)
    print("encoding execution time: %s seconds" % (time.time() - starttime))
    return df

In [27]:
# -------- Memory reduction function ---------
def reduce_memory(df):
    """Reduce memory usage of a dataframe by setting data types. """
    start_mem = df.memory_usage().sum() / 1024 ** 2
    print('Initial df memory usage is {:.2f} MB for {} columns'
          .format(start_mem, len(df.columns)))

    for col in df.columns:
        col_type = df[col].dtypes
        if col_type != object:
            cmin = df[col].min()
            cmax = df[col].max()
            if str(col_type)[:3] == 'int':
                # Can use unsigned int here too
                if cmin > np.iinfo(np.int8).min and cmax < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif cmin > np.iinfo(np.int16).min and cmax < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif cmin > np.iinfo(np.int32).min and cmax < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif cmin > np.iinfo(np.int64).min and cmax < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if cmin > np.finfo(np.float16).min and cmax < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif cmin > np.finfo(np.float32).min and cmax < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    memory_reduction = 100 * (start_mem - end_mem) / start_mem
    print('Final memory usage is: {:.2f} MB - decreased by {:.1f}%'.format(end_mem, memory_reduction))
    return df

In [28]:
# merge data set and flag difference and create difference to month ratio pipeline
# note this only takes care of 2 months for now
# if we want to merge 3 months then might need more work
def merge_and_flag(df_primary, df_secondary,  primary_month, secondary_month):
    months = primary_month - secondary_month
    month_difference = months*30.5
    month_1 = str(primary_month)
    month_2 = str(secondary_month)
    cols = df_primary.drop('身分證字號', axis=1).columns.to_list()
    df_total = pd.merge(df_primary, df_secondary, on='身分證字號', suffixes=('', '_' + month_2), how='left')
    for col in tqdm(cols):
        # slow as hell pandas operations, but too lazy to make vectorization, prolly do it some other time
        try:
            # flag difference between months
            df_total[col + month_1 + '_' + month_2 + '差異'] = df_total[col] - df_total[col + '_' + month_2]
            # dropp used previous month column to reduce dataframe size
            df_total.drop([col + '_' + month_2], axis=1, inplace=True)
            # create ratio of time by dividing by days of month difference
            df_total[col + month_1 + '_' + month_2 + '差異/' + str(month_difference)] = df_total[col + month_1 + '_' + month_2 + '差異'] / month_difference
        except:
            pass
    return df_total

In [29]:
# merge data set and flag difference and create difference to month ratio pipeline
# note this only takes care of 2 months for now
# if we want to merge 3 months then might need more work
def merge_and_flag_dict(df_primary, df_secondary,  primary_month, secondary_month):
    months = primary_month - secondary_month
    month_difference = months*30.5
    month_1 = str(primary_month)
    month_2 = str(secondary_month)
    cols = df_primary.drop('身分證字號', axis=1).columns.to_list()
    df_total = pd.merge(df_primary, df_secondary, on='身分證字號', suffixes=('', '_' + month_2), how='left')
    df_dict = df_total.to_dict()
    for col in tqdm(cols):
        try:
            # flag difference between months
            new_col = do_subtraction(df_dict[col], df_dict[col + '_' + month_2], col + month_1 + '_' + month_2 + '差異')
            df_dict.update(new_col)
            
            # dropp used previous month column to reduce dataframe size
            del df_dict[col + '_' + month_2]
            
            # create ratio of time by dividing by days of month difference
            df_total[col + month_1 + '_' + month_2 + '差異/' + str(month_difference)] = df_total[col + month_1 + '_' + month_2 + '差異'] / month_difference
        except:
            pass
    return df_total

In [30]:
def do_subtraction(dict1, dict2, colname="you forgot name bruh"):
    if len(dict1) == len(dict2):
        array1 = list(dict1.values())[0]
        array2 = list(dict2.values())[0]
        array3 = []
        if len(array1) == len(array2):
            for i in np.arange(0, len(array1)):
                value = array1[i] - array2[i]
                print(value)
                array3.append(value)
        else:
            print("ERROR: different number of rows")
    else:
        return print("dictionary is wrong lenth of keys, please check for errors")
    mydict = {colname:array3}
    return mydict

In [31]:
def do_month_ratio(dict, colname=)

SyntaxError: invalid syntax (<ipython-input-31-6a6fb83a25d6>, line 1)

In [32]:
# not used
# helper function to find all categorical
def find_cat_columns(df):
        # initialize our columns list
        wrongTypeCols = []
        multiclassCols = []
        # this list contains columns that we dont want to edit 
        # a.k.a columns that are type int64 yet has some linear relationship
        numericalCols = ['最近一個月預借現金次數',
                         '全行往來產品數(持有1階)',
                         '全行往來產品數(使用1階)',
                         '台幣活儲使用帳號數',
                         '外幣活期使用帳號數',
                         '身分證字號',  # this is not numerical, but this is our key to joining datasets, hence we dont want it to change
                         '客戶職稱',      # the following three should be combined as 1
                         'ELOAN職稱',
                         '信用卡職位別',
                        ]
        # for loop through entire dataframe
        for col in df.columns:
            if col not in numericalCols:
                # object columns
                # need to change type first, then decide if its binary or multiclass
                if (df[col].dtype == np.object):
                    wrongTypeCols.append(col)
                # correct type multiclass
                # note that some of these are actually numerical, but stored as int64 not float
                # need to take those out as well
                # lets set the threshhold to 7 multiclass at most, every other type of data will be considered numberical
                elif (2 < len(df[col].value_counts(dropna=False))) and (df[col].dtype == np.int64):
                    multiclassCols.append(col)
        return wrongTypeCols, multiclassCols

In [33]:
# LIGHTGBM CONFIGURATION AND HYPER-PARAMETERS
STRATIFIED_KFOLD = False
RANDOM_SEED = 737851
NUM_FOLDS = 10
EARLY_STOPPING = 100

LIGHTGBM_PARAMS = {
    'boosting_type': 'goss',
    'n_estimators': 10000,
#     'learning_rate': ,
#     'num_leaves': ,
#     'max_depth': ,
#     'subsample_for_bin': ,
#     'reg_alpha': ,
#     'reg_lambda': ,
#     'colsample_bytree': ,
#     'min_split_gain': ,
#     'subsample': 1,
    'is_unbalance': False,
    'silent':False,
    'verbose':-1
}

In [34]:
@contextmanager
def timer(name):
    t0 = time.time()
    yield
    print("{} - done in {:.0f}s".format(name, time.time() - t0))